In [1]:
import os
import shutil
import glob
import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/Colab폴더/train_label'
Batch_size = 7
train_input_shape = (227, 227, 3)



In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=['transistor-good', 'capsule-good', 'wood-good', 'bottle-good',
       'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good',
       'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good',
       'screw-thread_side', 'zipper-fabric_border', 'leather-good',
       'pill-scratch', 'toothbrush-good', 'hazelnut-crack',
       'screw-manipulated_front', 'zipper-good', 'tile-good',
       'carpet-good', 'metal_nut-good', 'bottle-contamination',
       'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined',
       'pill-color', 'screw-thread_top', 'cable-missing_cable',
       'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke',
       'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color',
       'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold',
       'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head',
       'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small',
       'leather-cut', 'cable-cut_outer_insulation',
       'zipper-squeezed_teeth', 'toothbrush-defective',
       'cable-cut_inner_insulation', 'pill-contamination',
       'cable-missing_wire', 'carpet-thread', 'grid-broken',
       'pill-faulty_imprint', 'hazelnut-hole', 'leather-glue',
       'leather-poke', 'transistor-damaged_case', 'wood-scratch',
       'tile-gray_stroke', 'capsule-faulty_imprint', 'grid-glue',
       'zipper-combined', 'carpet-color', 'grid-bent', 'pill-combined',
       'hazelnut-print', 'cable-combined', 'capsule-scratch',
       'metal_nut-bent', 'zipper-broken_teeth', 'tile-oil',
       'transistor-misplaced', 'grid-thread', 'grid-metal_contamination',
       'carpet-cut', 'wood-color', 'cable-cable_swap', 'tile-crack',
       'leather-color', 'cable-poke_insulation', 'transistor-cut_lead',
       'wood-hole', 'tile-rough', 'wood-liquid'],
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=Batch_size,
                                                    shuffle=True,
                                                    class_mode='categorical')

Found 4277 images belonging to 88 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(96, (11,11), strides=4, input_shape=(227,227,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))
model.add(BatchNormalization())

#Alexnet - 계층 2 : 5X5 필터를 256개 사용 , strides = 1, 활화화함수 = relu, 3x3 크기의 풀리계층 사용
model.add(ZeroPadding2D(2))
model.add(Conv2D(256,(5,5), strides=1, activation='relu'))

model.add(MaxPooling2D(pool_size=(3,3),strides=2))
model.add(BatchNormalization())

#Alexnet - 계층 3 : 3x3 필터를 384개 사용, strides =1 , 활성화함수 = relu
model.add(ZeroPadding2D(1))
model.add(Conv2D(384,(3,3), strides=1, activation='relu'))


#Alexnet - 계층 4 : 3x3 필터를 384개 사용, strides =1 , 활성화함수 = relu
model.add(ZeroPadding2D(1))
model.add(Conv2D(384,(3,3), strides=1, activation='relu'))


#Alexnet - 계층 5 : 3x3 필터를 256개 사용, strides =1 , 활성화함수 = relu, 3x3 크기의 풀리계층 사용
model.add(ZeroPadding2D(1))
model.add(Conv2D(256,(3,3), strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=2))


model.add(Flatten())

#Alexnet - 계층 6 : 4096개의 출력뉴런, 활성화함수 = relu
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

#Alexnet - 계층 7 : 4096게의 출력뉴런, 활성화함수 = relu
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

#Alexnet - 계층 8 : 1개의 출력뉴런, 활성화함수 = softmax

model.add(Dense(88, activation='softmax'))

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 batch_normalization_10 (Bat  (None, 27, 27, 96)       384       
 chNormalization)                                                
                                                                 
 zero_padding2d_8 (ZeroPaddi  (None, 31, 31, 96)       0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 27, 27, 256)       614656    
                                                      

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,    
                   steps_per_epoch=(4277 // 7),
                   epochs=20,)

model.save('/content/drive/MyDrive/Colab폴더/second.h5')

Epoch 1/20
611/611 [==============================] - 254s 414ms/step - loss: 1.6303 - accuracy: 0.7351
Epoch 2/20
611/611 [==============================] - 241s 394ms/step - loss: 1.0843 - accuracy: 0.8073
Epoch 3/20
611/611 [==============================] - 235s 385ms/step - loss: 0.9844 - accuracy: 0.8162
Epoch 4/20
611/611 [==============================] - 240s 392ms/step - loss: 0.9132 - accuracy: 0.8232
Epoch 5/20
611/611 [==============================] - 233s 382ms/step - loss: 0.8611 - accuracy: 0.8363
Epoch 6/20
611/611 [==============================] - 238s 389ms/step - loss: 0.8296 - accuracy: 0.8391
Epoch 7/20
611/611 [==============================] - 237s 388ms/step - loss: 0.8193 - accuracy: 0.8415
Epoch 8/20
611/611 [==============================] - 235s 384ms/step - loss: 0.7949 - accuracy: 0.8429
Epoch 9/20
611/611 [==============================] - 233s 382ms/step - loss: 0.7946 - accuracy: 0.8415
Epoch 10/20
611/611 [==============================] - 239s 391m

In [12]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab폴더/second.h5')

In [17]:
test_images = '/content/drive/MyDrive/Colab폴더/open/test/test/20001.png'

In [10]:
test_label =  ['transistor-good', 'capsule-good', 'wood-good', 'bottle-good',
       'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good',
       'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good',
       'screw-thread_side', 'zipper-fabric_border', 'leather-good',
       'pill-scratch', 'toothbrush-good', 'hazelnut-crack',
       'screw-manipulated_front', 'zipper-good', 'tile-good',
       'carpet-good', 'metal_nut-good', 'bottle-contamination',
       'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined',
       'pill-color', 'screw-thread_top', 'cable-missing_cable',
       'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke',
       'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color',
       'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold',
       'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head',
       'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small',
       'leather-cut', 'cable-cut_outer_insulation',
       'zipper-squeezed_teeth', 'toothbrush-defective',
       'cable-cut_inner_insulation', 'pill-contamination',
       'cable-missing_wire', 'carpet-thread', 'grid-broken',
       'pill-faulty_imprint', 'hazelnut-hole', 'leather-glue',
       'leather-poke', 'transistor-damaged_case', 'wood-scratch',
       'tile-gray_stroke', 'capsule-faulty_imprint', 'grid-glue',
       'zipper-combined', 'carpet-color', 'grid-bent', 'pill-combined',
       'hazelnut-print', 'cable-combined', 'capsule-scratch',
       'metal_nut-bent', 'zipper-broken_teeth', 'tile-oil',
       'transistor-misplaced', 'grid-thread', 'grid-metal_contamination',
       'carpet-cut', 'wood-color', 'cable-cable_swap', 'tile-crack',
       'leather-color', 'cable-poke_insulation', 'transistor-cut_lead',
       'wood-hole', 'tile-rough', 'wood-liquid']

In [16]:
test_loss, test_acc = model.evaluate(test_images,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)

IndexError: ignored

In [13]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [18]:
predictions = probability_model.predict(test_images)

IndexError: ignored